In [1]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

/root/miniconda3/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigbio/biored contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/biored
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [3]:
from text2knowledge.strategy1 import extract_concepts

import json

In [4]:
model_name = 'mistral:latest'

In [5]:
for data in datas['test']:
    pmid = data['pmid']
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    abstract_offset = passages[1]['offsets'][0]
    metadata = {
        'pmid': pmid,
        'type': 'abstract',
    }
    entities_extracted = extract_concepts(abstract, model=model_name, metadata=metadata)

    if entities_extracted:
        entities_str = json.dumps(entities_extracted, indent=4)
        print(entities_str)
    else:
        print("No entities found.")
    break

 [
   {
       "entity": "Congenital long QT syndrome (LQTS)",
       "importance": 5,
       "category": "Disease"
   },
   {
       "entity": "Na(v)1.5 cardiac sodium channel",
       "importance": 5,
       "category": "Protein"
   },
   {
       "entity": "G-->A substitution at codon 1763",
       "importance": 4,
       "category": "Mutation"
   },
   {
       "entity": "Valine (GTG)",
       "importance": 2,
       "category": "AminoAcid"
   },
   {
       "entity": "Methionine (ATG)",
       "importance": 2,
       "category": "AminoAcid"
   },
   {
       "entity": "LQTS-3",
       "importance": 4,
       "category": "Disease"
   },
   {
       "entity": "Transmembrane segment 6 of domain IV",
       "importance": 3,
       "category": "Anatomy"
   },
   {
       "entity": "Lidocaine",
       "importance": 3,
       "category": "Compound"
   },
   {
       "entity": "Mexiletine",
       "importance": 3,
       "category": "Compound"
   },
   {
       "entity": "tsA201 mammalian

In [6]:
entities

[{'text': ['SCN5A'],
  'offsets': [[8, 13]],
  'concept_id': '6331',
  'semantic_type_id': 'GeneOrGeneProduct'},
 {'text': ['long QT syndrome'],
  'offsets': [[56, 72]],
  'concept_id': 'D008133',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['tachycardia'],
  'offsets': [[97, 108]],
  'concept_id': 'D013610',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['bradycardia'],
  'offsets': [[109, 120]],
  'concept_id': 'D001919',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['long QT syndrome'],
  'offsets': [[144, 160]],
  'concept_id': 'D008133',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['LQTS'],
  'offsets': [[162, 166]],
  'concept_id': 'D008133',
  'semantic_type_id': 'DiseaseOrPhenotypicFeature'},
 {'text': ['patient'],
  'offsets': [[292, 299]],
  'concept_id': '9606',
  'semantic_type_id': 'OrganismTaxon'},
 {'text': ['bradycardia'],
  'offsets': [[311, 322]],
  'concept_id': 'D001919',
  'semantic_type_id': 

In [7]:
data

{'pmid': '15485686',
 'passages': [{'type': 'title',
   'text': ['A novel SCN5A mutation manifests as a malignant form of long QT syndrome with perinatal onset of tachycardia/bradycardia.'],
   'offsets': [[0, 121]]},
  {'type': 'abstract',
   'text': ['OBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG

In [8]:
abstract_offset

[122, 1815]

In [9]:
abstract

'OBJECTIVE: Congenital long QT syndrome (LQTS) with in utero onset of the rhythm disturbances is associated with a poor prognosis. In this study we investigated a newborn patient with fetal bradycardia, 2:1 atrioventricular block and ventricular tachycardia soon after birth. METHODS: Mutational analysis and DNA sequencing were conducted in a newborn. The 2:1 atrioventricular block improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine, which also controlled the ventricular tachycardia. RESULTS: A novel, spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel, with a G-->A substitution at codon 1763, which changed a valine (GTG) to a methionine (ATG). The proband was heterozygous but the mutation was absent in the parents and the sister. Expression of this mutant channel in tsA201 mammalian cells by site-directed mutagenesis revealed a persistent tetrodotoxin-sensitive but li

In [10]:
def highlight_entities_label(s, start_idx, entities, color='#66CC66'):
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = ""
    last_r = 0
    for entity in entities:
        l, r = entity['offsets'][0]
        if not l >= start_idx:
            continue
        l -= start_idx
        r -= start_idx
        final += s[last_r:l]
        final += left + s[l:r] + right
        last_r = r
    final += s[r:]
    return final

In [11]:
from IPython.display import HTML

label = highlight_entities_label(abstract, abstract_offset[0], entities)
HTML(label)

In [14]:
def highlight_entities_extracted(s, entities, color='#00CCFF'):
    import re
    left = f"<span style='background-color: {color}'>"
    right = "</span>"
    final = s[:]
    for entity in entities:
        entity_text = entity['entity']
        final = re.sub('(?i)'+re.escape(entity_text), lambda m: left+entity_text+right, final)
    return final
predicted = highlight_entities_extracted(abstract, entities_extracted)
HTML(predicted)

In [25]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [21]:
import re

In [22]:
re.sub('hello', 'bye', 'hello HeLLo HELLO', flags=re.IGNORECASE)

'bye bye bye'

In [23]:
re.sub('Congenital long QT syndrome (LQTS)', 'hello', 'Congenital long QT syndrome (LQTS)', flags=re.IGNORECASE)

'Congenital long QT syndrome (LQTS)'

In [28]:
confusion_matrix = {"test":[0,0,0]}
categories = []

def match_entity(entity_a, entity_b):
    return entity_a['category'] == entity_b['category'] and\
model.query(entity_a['entity'], top_k=1)[2] == model.query(entity_b['entity'], top_k=1)[2]

# def macro_avg_f1_score(ans, gt):
#     # for sample_ans, sample_gt in zip(ans, gt):
#         # for category in categories:
#             # get all entities in the category
#     # ans_entities = [entity['entity']
#     #                 for entity in sample_ans if entity['category'] == category]
#     ans_entities = [entity['entity'] for entity in ans]
#     # gt_entities = [entity['text']
#     #                for entity in sample_gt if entity['category'] == category]
#     gt_entities = [entity['text'][0] for entity in gt]
#     TP = len(set(ans_entities) & set(gt_entities))
#     FP = len(set(ans_entities) - set(gt_entities))
#     FN = len(set(gt_entities) - set(ans_entities))
#     confusion_matrix["test"][0] += TP
#     confusion_matrix["test"][1] += FP
#     confusion_matrix["test"][2] += FN
#     f1_score = []
#     # for category in categories:
#     TP, FP, FN = confusion_matrix["test"]
#     precision = TP/(TP+FP) if TP+FP else float('nan')
#     recall = TP/(TP+FN) if TP+FN else float('nan')
#     f1_score.append(2*(precision * recall) / (precision + recall))
#     return sum(f1_score) / len(f1_score)

# macro_avg_f1_score(entities_extracted, entities)

In [44]:
entities_extracted

[{'entity': 'Congenital long QT syndrome (LQTS)',
  'importance': 5,
  'category': 'Disease',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'In utero onset of rhythm disturbances',
  'importance': 3,
  'category': 'BiologicalProcess',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Newborn patient',
  'importance': 4,
  'category': 'Anatomy',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Fetal bradycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': '2:1 atrioventricular block',
  'importance': 4,
  'category': 'Pathway',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Ventricular tachycardia',
  'importance': 3,
  'category': 'Symptom',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Mutational analysis and DNA sequencing',
  'importance': 4,
  'category': 'MolecularFunction',
  'pmid': '15485686',
  'type': 'abstract'},
 {'entity': 'Lidocaine infusion',
  'importance': 3,
  

In [10]:
entities[1]

{'text': ['long QT syndrome'],
 'offsets': [[56, 72]],
 'concept_id': 'D008133',
 'semantic_type_id': 'DiseaseOrPhenotypicFeature'}

In [11]:
entities_extracted[0]

{'entity': 'Congenital long QT syndrome (LQTS)',
 'importance': 5,
 'category': 'Disease',
 'pmid': '15485686',
 'type': 'abstract'}

In [14]:
model.query(entities_extracted[0]['entity'], top_k=20)

(array([[0.97932327, 0.9789596 , 0.9789596 , 0.97796655, 0.9778704 ,
         0.97784853, 0.97774506, 0.9776753 , 0.9774991 , 0.97747386,
         0.9774325 , 0.97729903, 0.97713727, 0.9769972 , 0.9769947 ,
         0.97682893, 0.97670466, 0.97670466, 0.9763256 , 0.9752341 ]],
       dtype=float32),
 array([[  5550, 538983, 523573,  24920,   9590,  17247,   6439,  10193,
           9472,   9021,   4087,   1920,  10304,  13511,   5957,   5168,
          29175,  23463,  15586, 533892]]),
 [5550                  long QT syndrome 1
  538983       Long QT syndrome congenital
  523573       Long QT syndrome congenital
  24920        obsolete long QT syndrome 1
  9590                  long qt syndrome 8
  17247                 long QT syndrome 6
  6439                  long QT syndrome 3
  10193                 long QT syndrome 2
  9472                 long QT syndrome 13
  9021                 long QT syndrome 16
  4087                 long QT syndrome 11
  1920                 long QT syndr

In [15]:
model.query(entities[1]['text'], top_k=20)

(array([[1.0000001 , 1.        , 1.        , 1.        , 0.9940371 ,
         0.98827636, 0.97789556, 0.97636503, 0.9754645 , 0.9740887 ,
         0.9740887 , 0.9740887 , 0.97408867, 0.97390854, 0.9734761 ,
         0.97306156, 0.9729073 , 0.97290725, 0.9729072 , 0.972878  ]],
       dtype=float32),
 array([[534835, 448862,  11832, 486683,  21876,   2749,   9339,  30463,
          10193, 539013,  18950, 429186, 523834, 499481,   4606,   4001,
           3963, 535311, 461080,   5550]]),
 [534835               Long QT syndrome
  448862               Long QT Syndrome
  11832                long QT syndrome
  486683               Long QT syndrome
  21876               short QT syndrome
  2749        familial long QT syndrome
  9339                 Timothy syndrome
  30463         Long QT syndrome type 3
  10193              long QT syndrome 2
  539013               Brugada syndrome
  18950                Brugada syndrome
  429186               Brugada Syndrome
  523834               Brugad

In [24]:
model.query("atrioventricular block", top_k=10)

(array([[0.9999996 , 0.99999946, 0.99999946, 0.99999946, 0.98979944,
         0.9897993 , 0.98979926, 0.98644674, 0.9864467 , 0.9864467 ]],
       dtype=float32),
 array([[484683, 534046, 427038,  10136, 484687,  18606, 534047, 461241,
         488112,  23947]]),
 [484683           Atrioventricular block
  534046           Atrioventricular block
  427038           Atrioventricular Block
  10136            atrioventricular block
  484687    Atrioventricular dissociation
  18606     atrioventricular dissociation
  534047    Atrioventricular dissociation
  461241                 Sinoatrial Block
  488112                 Sinoatrial block
  23947                  sinoatrial block
  Name: name, dtype: object])

In [23]:
model.query("2:1 atrioventricular block", top_k=10)

(array([[0.9726196 , 0.97260904, 0.97260904, 0.97142196, 0.97102666,
         0.9710048 , 0.97037643, 0.97018284, 0.9698629 , 0.9698545 ]],
       dtype=float32),
 array([[492001, 536885, 499981,  22013, 520764, 520763, 506854, 489239,
         530397, 498891]]),
 [492001         Mobitz (type) II atrioventricular block
  536885            Atrioventricular block second degree
  499981            Atrioventricular block second degree
  22013            Mobitz type II atrioventricular block
  520764    Type II second degree atrioventricular block
  520763     Type I second degree atrioventricular block
  506854      Other second degree atrioventricular block
  489239                Atrioventricular block, complete
  530397               High grade atrioventricular block
  498891                  AV block complete with syncope
  Name: name, dtype: object])

In [46]:
model.query(entities[2]['text'], top_k=10)

(array([[1.0000005 , 1.0000005 , 1.0000004 , 1.0000004 , 0.98926437,
         0.9892642 , 0.9868732 , 0.9868732 , 0.98687315, 0.98687315]],
       dtype=float32),
 array([[463020,    791, 535387, 488295,    229, 534017, 486324, 444811,
         534699,    334]]),
 [463020    Tachycardia
  791       tachycardia
  535387    Tachycardia
  488295    Tachycardia
  229        arrhythmia
  534017     Arrhythmia
  486324    Hypotension
  444811    Hypotension
  534699    Hypotension
  334       hypotension
  Name: name, dtype: object])

In [10]:
set([j['semantic_type_id'] for i in datas["test"] for j in i['entities']])

{'CellLine',
 'ChemicalEntity',
 'DiseaseOrPhenotypicFeature',
 'GeneOrGeneProduct',
 'OrganismTaxon',
 'SequenceVariant'}

In [13]:
from text2knowledge.eval import BioBERT

model = BioBERT(
    model_name = 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext',
    index_path = '/root/text2knowledge/data/biovecs.index',
    db_path = '/root/text2knowledge/data/entities.tsv'
)

Loading model...


Some weights of the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading index...


/root/text2knowledge/text2knowledge/eval.py:20: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.entities = pd.read_csv(db_path, sep='\t')


In [2]:
model.query('Congenital long QT syndrome (LQTS)')

(array([[0.97932327, 0.9789596 , 0.9789596 , 0.97796655, 0.9778704 ]],
       dtype=float32),
 array([[  5550, 523573, 538983,  24920,   9590]]),
 [5550               long QT syndrome 1
  523573    Long QT syndrome congenital
  538983    Long QT syndrome congenital
  24920     obsolete long QT syndrome 1
  9590               long qt syndrome 8
  Name: name, dtype: object])

In [1]:
from text2knowledge.normalizer import Normalizer

In [2]:
class Arg:
    def __init__(self):
        # self.mention = "G-->A substitution at codon 1763"
        self.model_name_or_path = "dmis-lab/biosyn-biobert-bc2gn"
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = "/root/text2knowledge/data/dictionary/dict_Gene.txt"
        self.use_cuda = True
arg = Arg()
normalizer = Normalizer(arg)
normalizer.normalize("G-->A substitution at codon 1763")

TypeError: 'NoneType' object is not callable

In [7]:
normalizer.biosyn.embed_sparse(names=["G-->A substitution at codon 1763"])

TypeError: 'NoneType' object is not callable

In [2]:
from text2knowledge.normalizer import normalize

class Arg:
    def __init__(self):
        self.mention = "long QT syndrome (LQTS)"
        self.model_name_or_path = "dmis-lab/biosyn-biobert-ncbi-disease"
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = "/root/text2knowledge/data/dictionary/dict_Disease_20210630.txt"
        self.use_cuda = True
arg = Arg()
normalize(arg)



KeyboardInterrupt



In [1]:
from text2knowledge.normalizer import Normalizer

In [ ]:
class Arg:
    def __init__(self):
        # self.mention = "long QT syndrome (LQTS)"
        self.model_name_or_path = "dmis-lab/biosyn-biobert-ncbi-disease"
        self.show_embeddings = False
        self.show_predictions = True
        self.dictionary_path = "/root/text2knowledge/data/dictionary/dict_Disease_20210630.txt"
        self.use_cuda = True
arg = Arg()

disease_normalizer = Normalizer(arg)

In [3]:
disease_normalizer.normalize("long QT syndrome (LQTS)")

/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/file_download.py:678: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainabi

Loaded dictionary from cached file ./tmp/cached_biosyn-biobert-ncbi-disease_dict_Disease_20210630.pk


{'mention': 'long qt syndrome lqts',
 'predictions': [{'name': 'Long QT Syndrome', 'id': 'MESH:D008133'},
  {'name': 'Long Qt Syndrome 1/2', 'id': 'MESH:C566006'},
  {'name': 'Long Qt Syndrome 2|LONG QT SYNDROME 1/2, DIGENIC, INCLUDED;LQT1/2, DIGENIC, INCLUDED|LONG QT SYNDROME 2/3, DIGENIC, INCLUDED;LQT2/3, DIGENIC, INCLUDED|LONG QT SYNDROME 2/5, DIGENIC, INCLUDED;LQT2/5, DIGENIC, INCLUDED|LONG QT SYNDROME 2/9, DIGENIC, INCLUDED;LQT2/9, DIGENIC, INCLUDED|LONG QT SYNDROME 2, ACQUIRED, SUSCEPTIBILITY TO, INCLUDED|LQT2',
   'id': 'MESH:C563614'},
  {'name': 'Long Qt Syndrome 10', 'id': 'OMIM:611819,MESH:C567514'},
  {'name': 'LONG QT SYNDROME 13', 'id': 'OMIM:613485'}]}